## Installation de bibliotheques 

**pip install beautifulsoup4**

**pip install requests**


In [43]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
from datetime import datetime

## Scraper les articles bioRxiv liés à la bioinformatique 

In [44]:
url = 'https://www.biorxiv.org'



In [45]:
# creer base de données SQLite "bioinformatics_articles" 
database = sqlite3.connect('bioinformatics_articles.db')
c = database.cursor() #ici le curseur 'c' permettra d'executer/parcourir les requetes SQL sur database


c.execute("DROP TABLE IF EXISTS articles")

# create table 'articles' avec col id,title et le lien 
c.execute('''CREATE TABLE IF NOT EXISTS articles
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
             title TEXT, 
             link TEXT,
             doi TEXT UNIQUE,
             date TEXT,
             pdf_link TEXT,
             abstract TEXT)''')
#id c'est un identifiant unique généré automatiquement 

## Parcourir les articles  de la collection Bioinformatics contenant 2153 pages d'articles et enregistrer leurs informations

In [46]:

## Nombre de pages à scraper (exemple : 10 pages à la fois)
from pickle import TRUE


total_pages = 3

# boucle qui parcour chaque page de la collection Bioinformatics
for page in range(1, total_pages + 1):
    print(f"Scraping page {page} sur {total_pages}")

    # requete qui recup la page bioinfo actuelle
    response = requests.get(f'{url}/collection/bioinformatics?page={page}') #? mean numero de la {page}
    soup = BeautifulSoup(response.content, 'html.parser')

    # Trouver les liens vers les articles sur cette page
    articles = soup.find_all('a', class_='highwire-cite-linked-title')  # find liens hypertextes avec classe CSS

    print(f'Nombre total d\'articles trouvés sur la page {page} : {len(articles)}')

    # Parcourir tous les articles de la page et enregistrer leurs titres et liens
    for article in articles:
        title = article.text.strip()  # extraire le titre dans la balise <a>, accessible via .text.
        link = url + article['href']  # extraire le lien , href=destination du lien de mon article

        # Requête pour accéder à la page de l'article
        article_response = requests.get(link)
        article_soup = BeautifulSoup(article_response.content, 'html.parser')

        # extraire le DOI (dans une balise meta ou dans l'URL)
        doi_tag = article_soup.find('meta', {'name': 'citation_doi'})  # balise <meta> qui a l’attribut name="citation_doi".
        doi = doi_tag['content'] if doi_tag else 'DOI non disponible'  # si balise existe -> attribut content contient la valeur doi

        # extraire la date de publication (dans une balise meta ou time)et la formater au format ISO 8601
        date_tag = article_soup.find('meta', {'name': 'citation_publication_date'})
        date = date_tag['content'] if date_tag else 'Date non disponible'
        # Remplacer les barres obliques par des tirets avant la conversion
        if date != 'Date non disponible':
            date = date.replace('/', '-')  # Remplacer '/' par '-'
            date = datetime.strptime(date, '%Y-%m-%d').date().isoformat()

        # extraire le lien vers le fichier PDF
        # Trouver tous les liens et filtrer ceux qui contiennent 'PDF'
        pdf_links = article_soup.find_all('a')
        pdf_link = 'Lien PDF non disponible'
        for pdf in pdf_links:
            if pdf and 'PDF' in pdf.text:
                pdf_link = url + pdf['href']  # Construire le lien complet
                break  # Quitter la boucle une fois le lien trouvé
        
        
         # Extraire l'abstract
        abstract_tag = article_soup.find('meta', {'name': 'citation_abstract'})
        if abstract_tag:
            abstract_html = abstract_tag['content']
            clean_abstract = BeautifulSoup(abstract_html, "html.parser").get_text(strip=True)
        else:
            clean_abstract = 'Abstract non disponible'


        # tout inserer dans la base de données SQLite
        # Vérifier si l'article existe déjà dans la base de données (vérification sur DOI et date)
        c.execute("SELECT * FROM articles WHERE doi = ? AND date = ?", (doi, date))
        result = c.fetchone() # recupère la 1ere ligne du result donc si mm doi/date trouvé retourne cette ligne en tuple sinon NONE

        if result:
            print(f"L'article avec DOI {doi} existe déjà, ignoré.")
        else:
            # Insérer l'article dans la base de données
            c.execute("INSERT INTO articles (title, link, doi, date, pdf_link, abstract) VALUES (?, ?, ?, ?, ?, ?)", 
                      (title, link, doi, date, pdf_link, clean_abstract))
            print(f"Article ajouté : {title}")

        # Afficher les informations extraites
        print(f'Titre: {title}\nLien: {link}\nDOI: {doi}\nDate de publication: {date}\nLien PDF: {pdf_link}\nRésumé:{clean_abstract}\n')


        # pause d'une seconde pour éviter de surcharger le serveur
        time.sleep(1)
        
    # Sauvegarder les modifications dans la base de données après chaque page
    database.commit()
   


Scraping page 1 sur 3
Nombre total d'articles trouvés sur la page 1 : 10
Article ajouté : Enhanced prediction of protein functional identity through the integration of sequence and structural features
Titre: Enhanced prediction of protein functional identity through the integration of sequence and structural features
Lien: https://www.biorxiv.org/content/10.1101/2024.09.30.615718v1
DOI: 10.1101/2024.09.30.615718
Date de publication: 2024-01-01
Lien PDF: https://www.biorxiv.org/content/10.1101/2024.09.30.615718v1.full.pdf+html
Résumé:Although over 300 million protein sequences are registered in a reference sequence database, only 0.2% have experimentally determined functions. This suggests that many valuable proteins, potentially catalyzing novel enzymatic reactions, remain undiscovered among the vast number of function-unknown proteins. In this study, we developed a method to predict whether two proteins catalyze the same enzymatic reaction by analyzing sequence and structural similari

##  Affichage de mes articles dans la base de données

In [48]:
# affiche articles stockés dans database 
c.execute("SELECT * FROM articles")

rows = c.fetchall() #récupere tous les results de la requete d'avant =tuple(id,title,linke ect) de chaque article

print("=== Liste des articles enregistrés ===\n")

for row in rows:
    print(f"ID: {row[0]}")
    print(f"Titre: {row[1]}")
    print(f"Lien: {row[2]}")
    print(f"DOI: {row[3]}")
    print(f"Date: {row[4]}")
    print(f"Lien PDF: {row[5]}")
    print(f"Résumé: {row[6]}")  # Afficher l'abstract ici
    print("-" * 50)  # separateur visuel entre chaque article



print("Les titres et les liens des articles ont été enregistrés et affichés dans la base de données SQLite.")



=== Liste des articles enregistrés ===

ID: 1
Titre: Enhanced prediction of protein functional identity through the integration of sequence and structural features
Lien: https://www.biorxiv.org/content/10.1101/2024.09.30.615718v1
DOI: 10.1101/2024.09.30.615718
Date: 2024-01-01
Lien PDF: https://www.biorxiv.org/content/10.1101/2024.09.30.615718v1.full.pdf+html
Résumé: Although over 300 million protein sequences are registered in a reference sequence database, only 0.2% have experimentally determined functions. This suggests that many valuable proteins, potentially catalyzing novel enzymatic reactions, remain undiscovered among the vast number of function-unknown proteins. In this study, we developed a method to predict whether two proteins catalyze the same enzymatic reaction by analyzing sequence and structural similarities, utilizing structural models predicted by AlphaFold2. We performed pocket detection and domain decomposition for each structural model. The similarity between prote

In [49]:
# Fermer la connexion à la base de données (quand tu as finis)
#database.close()